# Advanced Optimization Lab2: Three-Player Game
AOpt - Fall 2024

## 0.1 Jupyter Notebook Quick Guide

### Installation
1. Ensure that Python and pip (Python’s package installer) are installed on your system.
2. Install Jupyter Notebook using pip: ```pip install notebook```

### Starting Jupyter Notebook

You can directly launch Jupyter Notebook using software like VScode or Pycharm or you can open a web version of Jupyter Notebook using the following steps:
1. Open a terminal or command prompt.
2. Navigate to the directory where you want to start your notebook: ```cd your_directory_path```
3. Run the command: ```jupyter notebook```
4. Jupyter Notebook will start in your default web browser.

### Using Jupyter Notebook
- **Running a Cell**: Type your Python code into a cell, and press the run icon on the left side of the cell to run it.
- **Saving Your Work**: Click on "File" > "Save and Checkpoint" to save your notebook.

You can try running the following cell to see if the environment configuration is successful:

In [ ]:
1+2

## 0.2 Homework Requirements and Grading Criteria

### Submitting The Notebook
1. In the notebook, there are some code blocks that need you to complete. **These areas that require your attention will be marked with 'TODO'**.
2. When you have completed the entire experiment, please ensure that all code block outputs are correct. Export this ipynb file as an HTML file and make sure that the code block outputs (such as images) are displayed correctly in the HTML file. 
3. Finally, **submit this HTML file along with your assignment**.

That's it! You're ready to use Jupyter Notebook.

### Score

- 10/40: <font color="#E43040">**Complete**</font> all the missing code blocks marked with 'TODO' and <font color="#E43040">**draw**</font> a reasonable comparison of the average total cost between Hedge and Optimistic Hedge (plot the curves of cost as the number of iterations increases for both algorithms). You can refer to the image below. <font color="#E43040">**Submit**</font> an HTML file with all the output results.
  <p align="center">
  <img src="ref_fig.png"/>
  </p>
- 20/40: In the **"1.2 Hedge and Optimistic Hedge"** section, correctly <font color="#E43040">**complete**</font> the one-step update formula for the Optimistic Hedge Algorithm.
- 30/40: Set the step-size of Optimistic Hedge to make its average total cost has reached <font color="#E43040"> **less than 1**</font> By the <font color="#E43040">**10000-th**</font> round.
- 40/40: Set the step-size of Optimistic Hedge to make its average total cost has reached <font color="#E43040"> **less than 1**</font> By the <font color="#E43040">**2000-th**</font> round.

## 0.3 Importing The Package
Please install the required packages in the following cell and run the cell below to import these packages. __Note that it is not allowed to add any additional packages or function__.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 1. Background

## 1.1 Problem Formulation

We consider a three-player game, where the strategies of three players are represented by $\bf x, y$ and $\bf z$. We consider the game repeated $T$ times. In round $t$, after all three players ***simultaneously*** submit their strategies $({\bf x}_t, {\bf y}_t, {\bf z}_t)$, each player's individual ***cost*** is calculated using their own cost function. For example, $\bf x$-player' cost function is denoted as ${\cal G}^{\bf x}:({\bf x,y,z})\mapsto\mathbb{R}$, and ${\cal G}^{\bf y}:({\bf x,y,z})\mapsto\mathbb{R}$ for $\bf y$-player, ${\cal G}^{\bf z}:({\bf x,y,z})\mapsto\mathbb{R}$ for $\bf z$-player.

Let ${\cal G}(\cdot)\triangleq{\cal G}^{\bf x}(\cdot)+{\cal G}^{\bf y}(\cdot)+{\cal G}^{\bf z}(\cdot)$ denote the total cost for the three players. In this problem, we focus on a quantity: the *average total cost* of all players, i.e., $\overline{\cal G}_T\triangleq\frac{1}{T}\sum_{t=1}^T {\cal G}({\bf x}_t, {\bf y}_t, {\bf z}_t)$.

To be more specific, assume that each player picks a mixed strategy from $\Delta_d$. Each player has her own tensor to measure cost, that is, ${\bf x}$-player has $G^{\bf x}\in[0,1]^{d\times d\times d}$, ${\bf y}$-player has $G^{\bf y}\in[0,1]^{d\times d\times d}$, and ${\bf z}$-player has $G^{\bf z}\in[0,1]^{d\times d\times d}$. For the tensor $G\in\mathbb{R}^{d\times d\times d}$ and three vectors ${\bf x,y,z}\in\mathbb{R}^d$, We abbreviate $\sum_{i=1}^d\sum_{j=1}^d\sum_{k=1}^d G_{i,j,k}{\bf x}_i {\bf y}_j {\bf z}_k$ as $G[{\bf x,y,z}]$. Then, the cost functions for the three players are specified as:

$$
{\cal G}^{\bf x}({\bf x,y,z}) \triangleq G^{\bf x}[{\bf x,y,z}],\quad {\cal G}^{\bf y}({\bf x,y,z}) \triangleq G^{\bf y}[{\bf x,y,z}],\quad{\cal G}^{\bf z}({\bf x,y,z}) \triangleq G^{\bf z}[{\bf x,y,z}].
$$

In round $t$, after the three players submit their strategies ${\bf x}_t,{\bf y}_t$ and ${\bf z}_t$, they can observe the gradient of their own cost functions, which is $\nabla_{\bf x} {\cal G}^{\bf x}({\bf x}_t, {\bf y}_t, {\bf z}_t)$ for ${\bf x}$-player, $\nabla_{\bf y} {\cal G}^{\bf y}({\bf x}_t, {\bf y}_t, {\bf z}_t)$ for ${\bf y}$-player, and $\nabla_{\bf z} {\cal G}^{\bf z}({\bf x}_t, {\bf y}_t, {\bf z}_t)$ for ${\bf z}$-player.

In this experiment, we conduct two games: one game will have all players using the OMD algorithm, while the other game will have all players using the Optimistic OMD algorithm that you have designed in Homework 2 Problem 1. We then compare the results of the two games.

## 1.2 Hedge and Optimistic Hedge

We have learned about OMD with NE-entropy that updates using

$$
\boldsymbol{p}_{t+1} = \operatorname{argmin}_{\boldsymbol{p}\in\Delta_d} \big\{ \eta\langle \boldsymbol{\ell}_t, \boldsymbol{p} \rangle + \operatorname{KL}(\boldsymbol{p}\| \boldsymbol{p}_t) \big\}
$$

with the loss vector $\boldsymbol{\ell}_t\in[0,1]^d$, is equal to Hedge update:

$$
p_{t+1,i} \propto p _{t,i} \exp(-\eta \ell_{t,i}), \quad \forall i\in[d]. \qquad\qquad(\star)
$$

In fact, the Optimistic OMD algorithm with NE-entropy, that is,
$$
\begin{cases}
\begin{align*}
\widehat{\boldsymbol{p}}_{t+1} &= \operatorname{argmin}_{\boldsymbol{p}\in\Delta_d} \big\{ \eta\langle \boldsymbol{\ell}_t, \boldsymbol{p} \rangle + \operatorname{KL}(\boldsymbol{p}\| \widehat{\boldsymbol{p}}_t) \big\} \\
\boldsymbol{p}_{t+1} &= \operatorname{argmin}_{\boldsymbol{p}\in\Delta_d} \big\{ \eta\langle \widehat{\boldsymbol{\ell}}_{t+1}, \boldsymbol{p} \rangle + \operatorname{KL}(\boldsymbol{p}\| \widehat{\boldsymbol{p}}_{t+1}) \big\}
\end{align*}
\end{cases}
$$

with the loss vector $\boldsymbol{\ell}_t\in[0,1]^d$ and the optimism $\widehat{\boldsymbol{\ell}}_{t+1}\in[0,1]^d$, can also be equivalent to an "Optimistic Hedge" algorithm with only **one-step** update.

<font color="#E43040"> **You need to refer to Hedge update $(\star)$, to provide the one-step update formula of Optimistic Hedge:**</font>

$$
\begin{align*}
p _{t+1,i} \propto \text{[your answer here]}, \quad \forall i\in[d].
\end{align*}
$$


# 2. Implementation

## 2.1 Load Data and Calculate Gradients

We define a class `Playground` that contains three matrices $G^{\mathbf{x}},G^{\mathbf{y}},G^{\mathbf{z}}$, and has a method `get_grads` of returning the corresponding cost function gradients, i.e., $\nabla_{\bf x} {\cal G}^{\bf x}({\bf x}, {\bf y}, {\bf z}),\nabla_{\bf y} {\cal G}^{\bf y}({\bf x}, {\bf y}, {\bf z})$ and $\nabla_{\bf z} {\cal G}^{\bf z}({\bf x}, {\bf y}, {\bf z})$, based on a given strategies $(\mathbf{x},\mathbf{y},\mathbf{z})$.  
<font color="#E43040"> **Please fill in the content for TODO in the code block below**. </font> 

In [ ]:
class Playground:
    def __init__(self, dim):
        self.Gx = np.loadtxt('data/Gx.txt', dtype=float).reshape(dim, dim, dim)
        self.Gy = np.loadtxt('data/Gy.txt', dtype=float).reshape(dim, dim, dim)
        self.Gz = np.loadtxt('data/Gz.txt', dtype=float).reshape(dim, dim, dim)

    def get_grads(self, strategies : list) -> list:
        """
        Calculating the gradients of cost functions for each player.

        :param strategies: list of strategies
        :return grads: list of gradients for each player
        """
        x, y, z = strategies
        grads = []
        # calculate the gradient
        # %%
        # TODO(1/5) your code here
        #
        #
        # %%
        return grads

## 2.2 Hedge Implementation

<font color="#E43040"> **Please fill in the content for TODO in the code block below** </font> to implement the single-step update part of the Hedge algorithm.

In [ ]:
class Hedge:
    """
    The vanilla online mirror descent in simplex, i.e., Hedge
    """
    def __init__(self, init_strategy, eta):
        self.p = init_strategy # optimization variable p
        self.eta = eta # constant step-size
        self.loss_list = []

    def one_step(self, grad) -> None:
        # record the loss at this iteration
        self.loss_list.append(np.dot(grad, self.p))

        # single-step update p
        # %%
        # TODO(2/5) your code here
        #
        #
        # %%
        return None

## 2.3 Optimistic Hedge Implementation

<font color="#E43040"> **Please fill in the content for TODO in the code block below** </font> to implement the single-step update part of the Optimistic Hedge algorithm.

In [ ]:
class OptHedge:
    """
    The optimistic online mirror descent in simplex, i.e., Optimistic Hedge
    """
    def __init__(self, init_strategy, eta):
        self.p = init_strategy # optimization variable p
        self.eta = eta # constant step-size
        self.optimism = np.zeros_like(self.p) # optimism
        self.loss_list = []

    def one_step(self, grad) -> None:
        # record the loss at this iteration
        self.loss_list.append(np.dot(grad, self.p))

        # single-step update p
        # %%
        # TODO(3/5) your code here
        #
        #
        # %%
        return None

# 3. Experiment

## 3.1 Parameters

In this experiment, we set dimension (`dim`) $d=5$, and game rounds $T=10^4$. We fix the constant step-size `eta_Hedge` of Hedge to $\sqrt{\frac{\ln d}{T}}$.  
<font color="#E43040"> **Please fill in the content for TODO in the code block below** </font> to set the constant step-size `eta_OptHedge` of Optimistic Hedge that you have designed in Homework 2 Problem 1.

In [ ]:
dim = 5 # fixed
T = 10000 # fixed
eta_Hedge = np.sqrt(np.log(dim) / T) # fixed

# %%
# TODO(4/5) your code here
eta_OptHedge = 0
# %%

## 3.2 Experiment and Result

Now you can start two three-player games! One game will have all players using the Hedge algorithm, while the other game will have all players using the Optimistic Hedge algorithm.

In [ ]:
playground = Playground(dim)
players_Hedge = [Hedge(np.ones(dim)/dim, eta_Hedge) for _ in range(3)]
players_OptHedge = [OptHedge(np.ones(dim)/dim, eta_OptHedge) for _ in range(3)]

# game by players using Hedge
for t in range(T):
    strategies = [players_Hedge[idx].p for idx in range(3)]
    grads = playground.get_grads(strategies)
    for idx in range(3):
        players_Hedge[idx].one_step(grads[idx])

# game by players using Optimistic Hedge
for t in range(T):
    strategies = [players_OptHedge[idx].p for idx in range(3)]
    grads = playground.get_grads(strategies)
    for idx in range(3):
        players_OptHedge[idx].one_step(grads[idx])

<font color="#E43040"> **Please fill in the content for TODO in the code block below** </font> to calculate the average total cost lists, i.e., $\{\overline{\mathcal{G}}_t\}_{t=1}^T$, for the two games.

In [ ]:
avg_total_cost_list_Hedge = np.zeros(T)
avg_total_cost_list_OptHedge = np.zeros(T)
# %%
# TODO(5/5) your code here
#
#
# %%

Run the following code cell to plot a comparison of the two games.

In [ ]:
fig = plt.figure(figsize=(15, 6))
for _ in range(3):
    ax = fig.add_subplot(2, 3, _+1)
    ax.set_xlabel('Iterations')
    ax.set_ylabel(f'Cost')
    ax.set_title(f'Cost of player {_+1}')
    ax.plot(range(T), players_Hedge[_].loss_list, label='Hedge')
    ax.plot(range(T), players_OptHedge[_].loss_list, label='Optimistic Hedge')
    ax.legend(loc=1)

ax = fig.add_subplot(2, 1, 2)
ax.set_xlabel('Iterations')
ax.set_ylabel(f'Cost')
ax.set_title(f'Average total cost of players')
ax.plot(range(T), avg_total_cost_list_Hedge, label='Hedge')
ax.plot(range(T), avg_total_cost_list_OptHedge, label='Optimistic Hedge')
ax.axhline(y=1, color='gray', linestyle='--')
plt.scatter(1999, avg_total_cost_list_OptHedge[1999], color='red', marker='o')
ax.legend(loc=1)

plt.tight_layout()
plt.show()

Congrats! You have successfully completed all the tasks for this experiment. If you have any suggestions for the experiment, feel free to share them.

# References

[1] *Advanced Optimization*. [Link](https://www.pengzhao-ml.com/course/AOpt2024fall/)